<a href="https://colab.research.google.com/github/Madhu-712/Customer-support-langgraph-bot/blob/main/LangGraph_order_management_chatbot_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install -U langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.9/130.9 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 3.2 MB/s eta 0:00:00


In [2]:
%pip install langchain_groq langchain groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 3.6 MB/s eta 0:00:00


In [3]:
%pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.0 MB/s eta 0:00:00


In [4]:
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

NameError: name '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' is not defined

In [6]:
import getpass
import os

if not os.environ.get("GROQ_API_KEY"):
    os.environ["GROQ_API_KEY"] = getpass.getpass("API key:\n")

In [7]:
from  langchain_groq import ChatGroq
llm = ChatGroq(model="Llama3-8b-8192")
#llm=ChatGroq(model_name="Gemma2-9b-It")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7f2fa5e766d0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7f2fa5e59dd0>, model_name='Llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [8]:

from langchain_core.tools import tool
import datetime

class OrderStatusManager:
    def __init__(self):
        self.order_statuses = {
            "12345": "Shipped",
            "67890": "Processing",
            "11223": "Delivered"
        }
        self.order_price = {
            "12345" : "500$",
            "67890" : "79$",
            "11223" : "900$"
        }
        self.order_shipped={
            "12345" : "22/2/25",
            "67890" : "24/2/25",
            "11223" : "2/2/25"
        }
           # Store original prices for resetting
        self.original_prices = self.order_price.copy()

    def get_total_price(self, order_id: str) -> str:
        """Calculates total sum  price for an order"""
        return self.orderprice.get(order_id,"total sum price")

    def get_order_status(self, order_id: str) -> str:
        """Fetches the status of a given order ID."""
        return self.order_statuses.get(order_id, "Order ID not found.")

    def initiate_return(self, order_id: str, reason: str) -> str:
        """Initiates a return for a given order ID with a specified reason.
        Return not initiated if shipping above 10 days.
        """
        if order_id in ["12345", "67890", "11223"]:
            # Get shipping date and calculate days since shipped
            shipping_date_str = self.order_shipped.get(order_id)

            if shipping_date_str:  # Check if shipping date exists for this order
                shipping_date = datetime.datetime.strptime(shipping_date_str, "%d/%m/%y").date()
                days_since_shipped = (datetime.date.today() - shipping_date).days  # Get current date

                if days_since_shipped > 10:
                    return f"Return cannot be initiated for order {order_id} as it has been more than 10 days since shipping."

            # Proceed with return initiation if within 10 days
            self.order_statuses[order_id] = "Return Initiated"
            # Calculate the penalty (2% deduction)
            original_price = float(self.order_price[order_id].rstrip("$"))
            penalty = original_price * 0.02
            new_price = original_price - penalty
            self.order_price[order_id] = f"{new_price:.2f}$"  # Update price with penalty
            return f"Return initiated for order {order_id} due to: {reason}. Price after penalty is {self.order_price[order_id]}."
        else:
            return "Order ID not found. Cannot initiate return."



    def cancel_return(self, order_id: str, reason: str) -> str:
        """Cancel return for an order ID by stating reason - changed mind."""
        if order_id in self.order_statuses:
            if self.order_statuses[order_id] == "Return Initiated":
                self.order_statuses[order_id] = self.order_statuses.get(order_id, "Cancelled Order") # Reset status (assuming original status is unknown)
                                                                                                     # you might want to store original status before return initiated.
                # Reset the price (remove penalty) - If you store original price use that instead.
                # assuming original price was order_price before penalty
                self.order_price[order_id] = self.order_price.get(order_id, "order_price") # Reset price

                return f"Return cancellation initiated for order {order_id} due to: {reason}. Price reset to {self.order_price[order_id]}."
            else:
                return f"Order {order_id} is not in 'Return Initiated' status. Cannot cancel return."
        else:
            return "Order ID not found. Cannot cancel return."

# Create an instance of OrderStatusManager
order_manager = OrderStatusManager()

# Wrap the methods as tools
get_total_price_tool = tool(order_manager.get_total_price)
get_order_status_tool = tool(order_manager.get_order_status)
initiate_return_tool = tool(order_manager.initiate_return)
cancel_return_tool = tool(order_manager.cancel_return) # Add the cancel_return tool

In [9]:
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage  # Add this line to import necessary classes
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage  # Add this line to import necessary classes

from langgraph.graph import MessagesState  # This is the crucial line to import MessagesState


from langgraph.graph import MessagesState
from langchain_core.messages import HumanMessage, ToolMessage
from langgraph.graph import START,END, StateGraph
from langgraph.prebuilt import tools_condition, ToolNode
from langchain_groq import ChatGroq
from langgraph.checkpoint.memory import MemorySaver  # Import MemorySaver

# Assuming you have order_manager, get_order_status_tool, initiate_return_tool defined

llm = ChatGroq(model="Llama3-8b-8192")

# Define your tools list with the updated tools
tools = [get_order_status_tool, initiate_return_tool,get_total_price_tool,cancel_return_tool]

llm_with_tools = llm.bind_tools(tools)

In [10]:
def assistant(state: MessagesState):
    """Use the LLM to decide the next step."""
    # Ensure llm_with_tools is defined and available in this scope
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

In [11]:

def check_return_conditions(state: MessagesState) -> str:
    """Decide whether to take action based on return conditions."""
    messages = state["messages"]

    # Check for return-related keywords and conditions
    for message in messages:
        if "return" in message.content.lower():
            # Check if ship date is within 10 days
            if "ship date" in message.content.lower():
                try:
                    # Attempt to extract ship date from the message
                    ship_date_str = message.content.lower().split("ship date")[1].strip()[:10]  # Assuming date format
                    ship_date = datetime.datetime.strptime(ship_date_str, "%Y-%m-%d").date()
                    days_since_shipped = (datetime.date.today() - ship_date).days

                    if days_since_shipped <= 10:
                        return "initiate_return"  # Route to initiate return node
                except (ValueError, IndexError):
                    pass  # Ignore if ship date extraction fails

            # Check if price update is mentioned
            if "price update" in message.content.lower():
                return "update_price"  # Route to update price node

    # Default: Proceed to END if no return conditions are met
    return END

In [12]:
# Graph
builder = StateGraph(MessagesState)
# Define nodes
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

# Define edges
builder.add_edge(START, "assistant")
builder.add_conditional_edges("assistant", tools_condition)
builder.add_conditional_edges("assistant", check_return_conditions)
builder.add_edge("tools", "assistant")

# Initialize memory to persist state between graph runs
checkpointer = MemorySaver()

In [13]:
# Compile the graph with the checkpointer
graph = builder.compile(checkpointer=checkpointer)  # Pass checkpointer here

In [14]:
# Now, run the workflow with the user input
user_input = "What is the current status of order 12345."
config = {"configurable": {"thread_id": "1"}}

events = graph.stream(
    {"messages": [("user", user_input)]}, config, stream_mode="values"
)

for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

What is the current status of order 12345.
================================== Ai Message ==================================
Tool Calls:
  get_order_status (call_h59v)
 Call ID: call_h59v
  Args:
    order_id: 12345
================================= Tool Message =================================
Name: get_order_status

Shipped
================================== Ai Message ==================================
Tool Calls:
  get_total_price (call_kxcd)
 Call ID: call_kxcd
  Args:
    order_id: 12345
================================= Tool Message =================================
Name: get_total_price

Error: AttributeError("'OrderStatusManager' object has no attribute 'orderprice'")
 Please fix your mistakes.
================================== Ai Message ==================================

I apologize for the mistake. Since the tool call id "call_kxcd" yielded an error, I will not proceed with that tool call. I

#let's check return condition with price update

In [15]:
# Now, run the workflow with the user input
user_input = "I want to  return the order 12345."
config = {"configurable": {"thread_id": "1"}}

events = graph.stream(
    {"messages": [("user", user_input)]}, config, stream_mode="values"
)

for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

I want to  return the order 12345.
================================== Ai Message ==================================
Tool Calls:
  initiate_return (call_5rnw)
 Call ID: call_5rnw
  Args:
    order_id: 12345
    reason: Changed my mind
================================= Tool Message =================================
Name: initiate_return

Return initiated for order 12345 due to: Changed my mind. Price after penalty is 490.00$.
================================== Ai Message ==================================
Tool Calls:
  cancel_return (call_j3dc)
 Call ID: call_j3dc
  Args:
    order_id: 12345
    reason: Changed my mind
================================= Tool Message =================================
Name: cancel_return

Return cancellation initiated for order 12345 due to: Changed my mind. Price reset to 490.00$.
================================== Ai Message ==================================

The order retu

#let's check cancel condition

In [16]:
user_input = " I want to cancel return for order 12345."
config = {"configurable": {"thread_id": "1"}}

events = graph.stream(
    {"messages": [("user", user_input)]}, config, stream_mode="values"
)

for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

 I want to cancel return for order 12345.
================================== Ai Message ==================================

The return for order 12345 has already been cancelled.


#let's check updated status of order

In [17]:

# Now, run the workflow with the user input
user_input = "What is the updated status of  order 12345 and total price."
config = {"configurable": {"thread_id": "1"}}

events = graph.stream(
    {"messages": [("user", user_input)]}, config, stream_mode="values"
)

for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

What is the updated status of  order 12345 and total price.
================================== Ai Message ==================================
Tool Calls:
  get_order_status (call_jhn2)
 Call ID: call_jhn2
  Args:
    order_id: 12345
================================= Tool Message =================================
Name: get_order_status

Return Initiated
================================== Ai Message ==================================
Tool Calls:
  get_order_status (call_m6sm)
 Call ID: call_m6sm
  Args:
    order_id: 12345
================================= Tool Message =================================
Name: get_order_status

Return Initiated
================================== Ai Message ==================================

The order with ID 12345 is still in the "Return Initiated" status.

However, the total price for the order is still $490.00.


#let's check return condition eligibility

In [18]:
# Now, run the workflow with the user input
user_input = "I want order 11223 to return.Is it eligible ?"
config = {"configurable": {"thread_id": "1"}}

events = graph.stream(
    {"messages": [("user", user_input)]}, config, stream_mode="values"
)

for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

I want order 11223 to return.Is it eligible ?
================================== Ai Message ==================================

I'll check the eligibility for return.

Since there is no information about the shipping date for order 11223, I'll assume it's not eligible for return.

Return not initiated for order 11223 due to: Shipping above 10 days.
